# Concise Implementation of Linear Regression

We will show you how to implement
the linear regression model
concisely by using high-level APIs

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Call upon the existing API in a framework to read data

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.4601,  0.0972],
         [-0.3190,  0.1226],
         [ 1.3400, -0.8231],
         [ 1.7629,  1.0843],
         [ 0.7487,  0.9190],
         [ 0.2896,  0.3263],
         [-0.0949,  0.9403],
         [-0.5657,  0.1407],
         [-0.2176, -1.3390],
         [-0.8592, -0.2787]]),
 tensor([[4.7901],
         [3.1476],
         [9.7068],
         [4.0478],
         [2.5764],
         [3.6767],
         [0.8345],
         [2.5750],
         [8.3238],
         [3.4502]])]

Use a framework's predefined layers

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

Initialize the model parameters

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

The `MSELoss` class computes the mean squared error, also known as squared $L_2$ norm

In [8]:
loss = nn.MSELoss()

Instantiate an `SGD` instance

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

The training loop itself is strikingly similar
to what we did when implementing everything from scratch

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000156
epoch 2, loss 0.000094
epoch 3, loss 0.000094


Compare the model parameters learned by training on finite data
and the actual parameters

In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([ 1.0097e-04, -9.7990e-05])
error in estimating b: tensor([-2.6703e-05])
